In [49]:

import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
import copy


## Read in generated features

In [3]:
article_embedding=pd.read_csv("/Users/wenxinzhang/Desktop/graduate/course/2nd_semester/AML/proj/final_dataset/article_embedding.csv")
article_embedding.head()


,Unnamed: 0,article_id,glove_vec0,glove_vec1,glove_vec2,glove_vec3,glove_vec4,glove_vec5,glove_vec6,glove_vec7,...,glove_vec90,glove_vec91,glove_vec92,glove_vec93,glove_vec94,glove_vec95,glove_vec96,glove_vec97,glove_vec98,glove_vec99
0,0,108775015,-0.257479,0.259222,0.136946,-0.252332,-0.042729,0.131018,-0.144289,0.243190,...,-0.027165,-0.103060,0.013883,-0.020606,-0.215009,0.026647,-0.232062,-0.323622,0.576949,0.144067
1,1,108775044,-0.263314,0.246323,0.143036,-0.237211,-0.055771,0.116141,-0.137286,0.251366,...,-0.040971,-0.110950,0.019565,-0.016827,-0.224636,0.025311,-0.231398,-0.318121,0.583604,0.140784
2,2,108775051,-0.240929,0.239273,0.203272,-0.333558,-0.046959,0.194002,-0.110387,0.294934,...,-0.101759,-0.036776,-0.033154,-0.059253,-0.293986,0.071189,-0.209148,-0.322547,0.592193,0.001066
3,3,110065001,-0.193816,0.283916,0.103505,-0.277805,-0.057416,0.158771,-0.079976,0.204655,...,-0.058738,-0.023422,0.001523,-0.092026,-0.162977,-0.039970,-0.210677,-0.233202,0.447961,0.058508
4,4,110065002,-0.195907,0.279292,0.105688,-0.272384,-0.062091,0.153438,-0.077466,0.207586,...,-0.063687,-0.026250,0.003560,-0.090671,-0.166428,-0.040449,-0.210439,-0.231230,0.450347,0.057331


In [4]:
del article_embedding[article_embedding.columns[0]]
article_embedding.head()

,article_id,glove_vec0,glove_vec1,glove_vec2,glove_vec3,glove_vec4,glove_vec5,glove_vec6,glove_vec7,glove_vec8,...,glove_vec90,glove_vec91,glove_vec92,glove_vec93,glove_vec94,glove_vec95,glove_vec96,glove_vec97,glove_vec98,glove_vec99
0,108775015,-0.257479,0.259222,0.136946,-0.252332,-0.042729,0.131018,-0.144289,0.243190,-0.137606,...,-0.027165,-0.103060,0.013883,-0.020606,-0.215009,0.026647,-0.232062,-0.323622,0.576949,0.144067
1,108775044,-0.263314,0.246323,0.143036,-0.237211,-0.055771,0.116141,-0.137286,0.251366,-0.134056,...,-0.040971,-0.110950,0.019565,-0.016827,-0.224636,0.025311,-0.231398,-0.318121,0.583604,0.140784
2,108775051,-0.240929,0.239273,0.203272,-0.333558,-0.046959,0.194002,-0.110387,0.294934,-0.147994,...,-0.101759,-0.036776,-0.033154,-0.059253,-0.293986,0.071189,-0.209148,-0.322547,0.592193,0.001066
3,110065001,-0.193816,0.283916,0.103505,-0.277805,-0.057416,0.158771,-0.079976,0.204655,-0.114611,...,-0.058738,-0.023422,0.001523,-0.092026,-0.162977,-0.039970,-0.210677,-0.233202,0.447961,0.058508
4,110065002,-0.195907,0.279292,0.105688,-0.272384,-0.062091,0.153438,-0.077466,0.207586,-0.113338,...,-0.063687,-0.026250,0.003560,-0.090671,-0.166428,-0.040449,-0.210439,-0.231230,0.450347,0.057331


In [5]:
x_item=article_embedding.drop(['article_id'],axis = 1)

Use KMeans to classify the products into 50 different groups.

In [6]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=50, random_state=0).fit(x_item)

In [7]:
d = {'article_id':article_embedding['article_id'], 'group': kmeans.labels_}
item_df=pd.DataFrame(data=d)
item_df.head()

,article_id,group
0,108775015,22
1,108775044,22
2,108775051,40
3,110065001,12
4,110065002,12


In [8]:
frequently_bought_item_id_count=pd.read_csv("/Users/wenxinzhang/Desktop/graduate/course/2nd_semester/AML/proj/final_dataset/frequently_bought_item_id_count.csv")
frequently_bought_item_id_count.head()

,Unnamed: 0,customer_id,article_id,count,t_dat
0,0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,797065001,2,2019-09-28
1,1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,811835004,4,2020-04-22
2,2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,750424014,2,2020-04-18
3,3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,742079001,1,2019-06-09
4,4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,399061015,2,2019-10-09


In [9]:
del frequently_bought_item_id_count[frequently_bought_item_id_count.columns[0]]
frequently_bought_item_id_count=frequently_bought_item_id_count.drop(['t_dat'],axis = 1)
frequently_bought_item_id_count.head()

,customer_id,article_id,count
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,797065001,2
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,811835004,4
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,750424014,2
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,742079001,1
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,399061015,2


Add product's corresponding group 

In [10]:
most_item=frequently_bought_item_id_count.merge(item_df, on='article_id',how='left')
most_item=most_item.rename(columns={"group": "most_group"})
most_item.head()

,customer_id,article_id,count,most_group
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,797065001,2,10
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,811835004,4,12
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,750424014,2,2
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,742079001,1,37
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,399061015,2,3


In [11]:
customer_features=pd.read_csv("/Users/wenxinzhang/Desktop/graduate/course/2nd_semester/AML/proj/final_dataset/customer_features.csv")
customer_features.head()


,Unnamed: 0,customer_id,visits,no_unique_items_bought,total_items_bought,list_of_items_bought,total_monetary_value,max_spent_on_item,recency,latency,avg_bought_per_visit
0,0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,10,19,21,"[625548001, 176209023, 627759010, 697138006, 5...",0.648983,0.054220,17,68.666667,2.100000
1,1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,23,64,86,"[583558001, 639677008, 640244003, 521269001, 6...",2.601932,0.084729,76,29.818182,3.739130
2,2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,7,14,18,"[663713001, 541518023, 663713001, 578020002, 3...",0.704780,0.067780,7,121.000000,2.571429
3,3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,1,2,2,"[732413001, 742079001]",0.060983,0.030492,471,0.000000,2.000000
4,4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,6,12,13,"[634249005, 677049001, 698286003, 707704003, 3...",0.469695,0.059305,41,134.000000,2.166667


In [12]:
del customer_features[customer_features.columns[0]]
customer_features.head()

,customer_id,visits,no_unique_items_bought,total_items_bought,list_of_items_bought,total_monetary_value,max_spent_on_item,recency,latency,avg_bought_per_visit
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,10,19,21,"[625548001, 176209023, 627759010, 697138006, 5...",0.648983,0.054220,17,68.666667,2.100000
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,23,64,86,"[583558001, 639677008, 640244003, 521269001, 6...",2.601932,0.084729,76,29.818182,3.739130
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,7,14,18,"[663713001, 541518023, 663713001, 578020002, 3...",0.704780,0.067780,7,121.000000,2.571429
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,1,2,2,"[732413001, 742079001]",0.060983,0.030492,471,0.000000,2.000000
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,6,12,13,"[634249005, 677049001, 698286003, 707704003, 3...",0.469695,0.059305,41,134.000000,2.166667


Find the most frequently purchasing group for each customer, name the column as label.

In [13]:
# convert article_ID to group number
def convert_to_str(df):
    purchasing_list=[]
    for i in range(len(df)):
        a=df['list_of_items_bought'][i][1:-1].split(", ")
        purchasing_list.append(a)
    return purchasing_list

purchasing_list=convert_to_str(customer_features)

In [14]:
item_group_dict=dict(zip(item_df['article_id'],item_df['group']))

In [15]:
purchasing_list_group = []
purchasing = copy.deepcopy(purchasing_list)
for i in purchasing:
    c = [item_group_dict[int(j)] if int(j) in item_group_dict else 0 for j in i]
    purchasing_list_group.append(c)

In [16]:
label=[]
for i in range(len(purchasing_list_group)):
    a=max(purchasing_list_group[i],key=purchasing_list_group[i].count)
    label.append(a)

In [17]:
customer_features['label']=label
customer_features.head()

,customer_id,visits,no_unique_items_bought,total_items_bought,list_of_items_bought,total_monetary_value,max_spent_on_item,recency,latency,avg_bought_per_visit,label
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,10,19,21,"[625548001, 176209023, 627759010, 697138006, 5...",0.648983,0.054220,17,68.666667,2.100000,10
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,23,64,86,"[583558001, 639677008, 640244003, 521269001, 6...",2.601932,0.084729,76,29.818182,3.739130,6
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,7,14,18,"[663713001, 541518023, 663713001, 578020002, 3...",0.704780,0.067780,7,121.000000,2.571429,12
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,1,2,2,"[732413001, 742079001]",0.060983,0.030492,471,0.000000,2.000000,37
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,6,12,13,"[634249005, 677049001, 698286003, 707704003, 3...",0.469695,0.059305,41,134.000000,2.166667,12


In [18]:
recent_transactio_article_id=pd.read_csv("/Users/wenxinzhang/Desktop/graduate/course/2nd_semester/AML/proj/final_dataset/recent_transactio_article_id.csv")
recent_transactio_article_id.head()

,Unnamed: 0,customer_id,recent_transaction_article_id
0,31788323,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,898573003
1,31788270,ff57364873464edd79e4807350b1cc0902d14f24490e48...,706016003
2,31788266,ff54fe15368409b7d0d691cdeebe2d6a3fee5178105277...,886566001
3,31788275,ff5b8a8b26bf93a66290e9bd1b73393ac6a58968a78519...,913597001
4,31788291,ff94f31e864d9b655643ac4d2adab3611c7241adb5d34c...,901666001


In [19]:
del recent_transactio_article_id[recent_transactio_article_id.columns[0]]
recent_transactio_article_id.head()

,customer_id,recent_transaction_article_id
0,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,898573003
1,ff57364873464edd79e4807350b1cc0902d14f24490e48...,706016003
2,ff54fe15368409b7d0d691cdeebe2d6a3fee5178105277...,886566001
3,ff5b8a8b26bf93a66290e9bd1b73393ac6a58968a78519...,913597001
4,ff94f31e864d9b655643ac4d2adab3611c7241adb5d34c...,901666001


Add product's corresponding group

In [20]:
recent_transactio_article_id=recent_transactio_article_id.rename(columns={"recent_transaction_article_id": "article_id"})
recent_transactio_article_id=recent_transactio_article_id.merge(item_df, on='article_id',how='left')
recent_transactio_article_id=recent_transactio_article_id.rename(columns={"group": "recent_group"})
recent_transactio_article_id

,customer_id,article_id,recent_group
0,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,898573003,38
1,ff57364873464edd79e4807350b1cc0902d14f24490e48...,706016003,28
2,ff54fe15368409b7d0d691cdeebe2d6a3fee5178105277...,886566001,32
3,ff5b8a8b26bf93a66290e9bd1b73393ac6a58968a78519...,913597001,26
4,ff94f31e864d9b655643ac4d2adab3611c7241adb5d34c...,901666001,27
...,...,...,...
1362276,0021da829b898f82269fc51feded4eac2129058ee95bd7...,625229004,30
1362277,0389e2e5ce41e5886aa49084beec73660c0092cd8d54be...,673776004,36
1362278,035f7a644366318ba089234153ff965dac19d68c607a9b...,507910001,23
1362279,0333d9ba2d726a88e105f58b664d0454ee8c3eb276c033...,666638003,40


In [22]:
customers=pd.read_csv("/Users/wenxinzhang/Desktop/graduate/course/2nd_semester/AML/proj/h-and-m-personalized-fashion-recommendations/customers.csv")
customers.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [23]:
# replace NA for FN and Active as 0 accroding to the meaing of these NA value.
customers=customers.fillna(value={'FN':0, 'Active':0})
customers['fashion_news_frequency']=customers['fashion_news_frequency'].replace('NONE','None')

In [24]:
customers.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0.0,0.0,ACTIVE,None,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0.0,0.0,ACTIVE,None,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0.0,0.0,ACTIVE,None,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0.0,0.0,ACTIVE,None,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


# Merge features

In [25]:
# merge customer purchasing info with most frequently bought item
merge1=customer_features.merge(most_item, on='customer_id')
merge1.head()

,customer_id,visits,no_unique_items_bought,total_items_bought,list_of_items_bought,total_monetary_value,max_spent_on_item,recency,latency,avg_bought_per_visit,label,article_id,count,most_group
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,10,19,21,"[625548001, 176209023, 627759010, 697138006, 5...",0.648983,0.054220,17,68.666667,2.100000,10,797065001,2,10
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,23,64,86,"[583558001, 639677008, 640244003, 521269001, 6...",2.601932,0.084729,76,29.818182,3.739130,6,811835004,4,12
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,7,14,18,"[663713001, 541518023, 663713001, 578020002, 3...",0.704780,0.067780,7,121.000000,2.571429,12,750424014,2,2
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,1,2,2,"[732413001, 742079001]",0.060983,0.030492,471,0.000000,2.000000,37,742079001,1,37
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,6,12,13,"[634249005, 677049001, 698286003, 707704003, 3...",0.469695,0.059305,41,134.000000,2.166667,12,399061015,2,3


In [26]:
# merge merge1 with most recent bought item
merge2=merge1.merge(recent_transactio_article_id, on='customer_id')
merge2.head()

,customer_id,visits,no_unique_items_bought,total_items_bought,list_of_items_bought,total_monetary_value,max_spent_on_item,recency,latency,avg_bought_per_visit,label,article_id_x,count,most_group,article_id_y,recent_group
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,10,19,21,"[625548001, 176209023, 627759010, 697138006, 5...",0.648983,0.054220,17,68.666667,2.100000,10,797065001,2,10,568601043,10
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,23,64,86,"[583558001, 639677008, 640244003, 521269001, 6...",2.601932,0.084729,76,29.818182,3.739130,6,811835004,4,12,826211002,23
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,7,14,18,"[663713001, 541518023, 663713001, 578020002, 3...",0.704780,0.067780,7,121.000000,2.571429,12,750424014,2,2,794321007,10
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,1,2,2,"[732413001, 742079001]",0.060983,0.030492,471,0.000000,2.000000,37,742079001,1,37,732413001,37
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,6,12,13,"[634249005, 677049001, 698286003, 707704003, 3...",0.469695,0.059305,41,134.000000,2.166667,12,399061015,2,3,791587015,37


In [27]:
# merge merge2 with customer info
merge3=merge2.merge(customers, on='customer_id')
merge3.head()

,customer_id,visits,no_unique_items_bought,total_items_bought,list_of_items_bought,total_monetary_value,max_spent_on_item,recency,latency,avg_bought_per_visit,...,count,most_group,article_id_y,recent_group,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,10,19,21,"[625548001, 176209023, 627759010, 697138006, 5...",0.648983,0.054220,17,68.666667,2.100000,...,2,10,568601043,10,0.0,0.0,ACTIVE,None,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,23,64,86,"[583558001, 639677008, 640244003, 521269001, 6...",2.601932,0.084729,76,29.818182,3.739130,...,4,12,826211002,23,0.0,0.0,ACTIVE,None,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,7,14,18,"[663713001, 541518023, 663713001, 578020002, 3...",0.704780,0.067780,7,121.000000,2.571429,...,2,2,794321007,10,0.0,0.0,ACTIVE,None,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,1,2,2,"[732413001, 742079001]",0.060983,0.030492,471,0.000000,2.000000,...,1,37,732413001,37,0.0,0.0,ACTIVE,None,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,6,12,13,"[634249005, 677049001, 698286003, 707704003, 3...",0.469695,0.059305,41,134.000000,2.166667,...,2,3,791587015,37,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [28]:
features = merge3.drop(['article_id_x','article_id_y','list_of_items_bought'],axis = 1)
features.head()

,customer_id,visits,no_unique_items_bought,total_items_bought,total_monetary_value,max_spent_on_item,recency,latency,avg_bought_per_visit,label,count,most_group,recent_group,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,10,19,21,0.648983,0.054220,17,68.666667,2.100000,10,2,10,10,0.0,0.0,ACTIVE,None,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,23,64,86,2.601932,0.084729,76,29.818182,3.739130,6,4,12,23,0.0,0.0,ACTIVE,None,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,7,14,18,0.704780,0.067780,7,121.000000,2.571429,12,2,2,10,0.0,0.0,ACTIVE,None,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,1,2,2,0.060983,0.030492,471,0.000000,2.000000,37,1,37,37,0.0,0.0,ACTIVE,None,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,6,12,13,0.469695,0.059305,41,134.000000,2.166667,12,2,3,37,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [29]:
features['club_member_status'] = features.club_member_status.astype("category").cat.codes
features['fashion_news_frequency'] = features.fashion_news_frequency.astype("category").cat.codes
features['postal_code'] = features.postal_code.astype("category").cat.codes
features.head()


,customer_id,visits,no_unique_items_bought,total_items_bought,total_monetary_value,max_spent_on_item,recency,latency,avg_bought_per_visit,label,count,most_group,recent_group,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,10,19,21,0.648983,0.054220,17,68.666667,2.100000,10,2,10,10,0.0,0.0,0,1,49.0,112784
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,23,64,86,2.601932,0.084729,76,29.818182,3.739130,6,4,12,23,0.0,0.0,0,1,25.0,57210
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,7,14,18,0.704780,0.067780,7,121.000000,2.571429,12,2,2,10,0.0,0.0,0,1,24.0,138919
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,1,2,2,0.060983,0.030492,471,0.000000,2.000000,37,1,37,37,0.0,0.0,0,1,54.0,128309
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,6,12,13,0.469695,0.059305,41,134.000000,2.166667,12,2,3,37,1.0,1.0,0,2,52.0,52277


# Build model

### Assumption:

Use the group of products which is occured most frequently as label. Which means assume customers will more likely to buy the product from that group.

In [30]:
features_update=features.dropna()
#sample = features_update.sample(frac=0.5, replace=False, random_state=1)
#len(sample)

In [31]:
X = features_update.drop(['label'],axis = 1)
y = features_update['label']
X.head()

,customer_id,visits,no_unique_items_bought,total_items_bought,total_monetary_value,max_spent_on_item,recency,latency,avg_bought_per_visit,count,most_group,recent_group,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,10,19,21,0.648983,0.054220,17,68.666667,2.100000,2,10,10,0.0,0.0,0,1,49.0,112784
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,23,64,86,2.601932,0.084729,76,29.818182,3.739130,4,12,23,0.0,0.0,0,1,25.0,57210
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,7,14,18,0.704780,0.067780,7,121.000000,2.571429,2,2,10,0.0,0.0,0,1,24.0,138919
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,1,2,2,0.060983,0.030492,471,0.000000,2.000000,1,37,37,0.0,0.0,0,1,54.0,128309
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,6,12,13,0.469695,0.059305,41,134.000000,2.166667,2,3,37,1.0,1.0,0,2,52.0,52277


In [32]:
X.corr()

,visits,no_unique_items_bought,total_items_bought,total_monetary_value,max_spent_on_item,recency,latency,avg_bought_per_visit,count,most_group,recent_group,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
visits,1.000000,0.850523,0.840325,0.801093,0.459309,-0.414285,-0.083690,-0.001652,0.370968,0.015888,-0.003394,0.135807,0.140071,-0.123651,0.137852,0.007221,0.050807
no_unique_items_bought,0.850523,1.000000,0.984752,0.939217,0.473740,-0.375006,-0.064200,0.257485,0.370727,0.017729,-0.000489,0.102745,0.106967,-0.111761,0.107392,-0.018324,0.046355
total_items_bought,0.840325,0.984752,1.000000,0.958660,0.463529,-0.357692,-0.065916,0.277920,0.441939,0.016972,-0.001209,0.097105,0.101288,-0.104876,0.101773,-0.014914,0.044999
total_monetary_value,0.801093,0.939217,0.958660,1.000000,0.550022,-0.329649,-0.062816,0.253570,0.408699,0.004072,-0.009281,0.083869,0.088118,-0.091849,0.088644,0.005126,0.041805
max_spent_on_item,0.459309,0.473740,0.463529,0.550022,1.000000,-0.270196,0.039416,0.113418,0.228746,-0.042975,-0.049769,0.059414,0.063927,-0.061035,0.060521,0.066162,0.039830
recency,-0.414285,-0.375006,-0.357692,-0.329649,-0.270196,1.000000,-0.257546,0.001619,-0.205004,0.004309,0.012069,-0.099125,-0.109693,0.181368,-0.106003,0.098210,-0.062405
latency,-0.083690,-0.064200,-0.065916,-0.062816,0.039416,-0.257546,1.000000,0.020302,-0.007759,0.000857,0.002385,0.031768,0.034485,-0.017006,0.033252,-0.011804,0.044540
avg_bought_per_visit,-0.001652,0.257485,0.277920,0.253570,0.113418,0.001619,0.020302,1.000000,0.265255,0.016004,0.013161,-0.034175,-0.032999,0.005359,-0.027619,-0.032067,0.004940
count,0.370968,0.370727,0.441939,0.408699,0.228746,-0.205004,-0.007759,0.265255,1.000000,0.037864,0.006996,0.039990,0.043077,-0.047146,0.044171,-0.004175,0.037580
most_group,0.015888,0.017729,0.016972,0.004072,-0.042975,0.004309,0.000857,0.016004,0.037864,1.000000,0.459724,0.000856,0.000726,-0.002600,0.001529,0.002377,0.000442


In [36]:
X_update= X.drop(['visits','no_unique_items_bought','FN','Active','total_monetary_value'],axis = 1)

In [37]:
X_update.corr()

,total_items_bought,max_spent_on_item,recency,latency,avg_bought_per_visit,count,most_group,recent_group,club_member_status,fashion_news_frequency,age,postal_code
total_items_bought,1.000000,0.463529,-0.357692,-0.065916,0.277920,0.441939,0.016972,-0.001209,-0.104876,0.101773,-0.014914,0.044999
max_spent_on_item,0.463529,1.000000,-0.270196,0.039416,0.113418,0.228746,-0.042975,-0.049769,-0.061035,0.060521,0.066162,0.039830
recency,-0.357692,-0.270196,1.000000,-0.257546,0.001619,-0.205004,0.004309,0.012069,0.181368,-0.106003,0.098210,-0.062405
latency,-0.065916,0.039416,-0.257546,1.000000,0.020302,-0.007759,0.000857,0.002385,-0.017006,0.033252,-0.011804,0.044540
avg_bought_per_visit,0.277920,0.113418,0.001619,0.020302,1.000000,0.265255,0.016004,0.013161,0.005359,-0.027619,-0.032067,0.004940
count,0.441939,0.228746,-0.205004,-0.007759,0.265255,1.000000,0.037864,0.006996,-0.047146,0.044171,-0.004175,0.037580
most_group,0.016972,-0.042975,0.004309,0.000857,0.016004,0.037864,1.000000,0.459724,-0.002600,0.001529,0.002377,0.000442
recent_group,-0.001209,-0.049769,0.012069,0.002385,0.013161,0.006996,0.459724,1.000000,0.000707,-0.000653,-0.000014,0.001665
club_member_status,-0.104876,-0.061035,0.181368,-0.017006,0.005359,-0.047146,-0.002600,0.000707,1.000000,-0.147293,0.080111,0.024441
fashion_news_frequency,0.101773,0.060521,-0.106003,0.033252,-0.027619,0.044171,0.001529,-0.000653,-0.147293,1.000000,0.025467,0.007581


In [38]:
X_train, X_test, y_train, y_test = train_test_split(X_update, y, test_size=0.2,random_state=0)
X_train_cid=X_train['customer_id']
X_test_cid=X_test['customer_id']
X_train=X_train.drop(['customer_id'],axis=1)
X_test=X_test.drop(['customer_id'],axis=1)
X_train.head()

,total_items_bought,max_spent_on_item,recency,latency,avg_bought_per_visit,count,most_group,recent_group,club_member_status,fashion_news_frequency,age,postal_code
909308,25,0.050831,618,25.333333,6.250000,3,49,49,0,1,27.0,132947
1029302,132,0.167797,42,26.038462,4.888889,4,35,19,0,1,22.0,115024
1300670,22,0.084729,57,58.200000,2.000000,2,10,16,0,2,31.0,245381
865176,10,0.084729,275,93.750000,2.000000,2,40,10,0,2,56.0,326016
218963,8,0.050831,197,150.500000,2.666667,1,23,22,0,2,44.0,65146


In [52]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(128,64,32), activation='relu', solver='adam', max_iter=500)
mlp.fit(X_train,y_train)

MLPClassifier(hidden_layer_sizes=(128, 64, 32), max_iter=500)

In [40]:
from sklearn.metrics import accuracy_score
pred_y = mlp.predict(X_test)
ac = accuracy_score(pred_y, y_test)
ac

0.10918887205537237

# Assumption:
Use the group of products which is most recently purchased as label. Which means assume customers will more likely to buy the product from that group.

In [41]:
X = features_update.drop(['recent_group'],axis = 1)
y = features_update['recent_group']
X.head()

,customer_id,visits,no_unique_items_bought,total_items_bought,total_monetary_value,max_spent_on_item,recency,latency,avg_bought_per_visit,label,count,most_group,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,10,19,21,0.648983,0.054220,17,68.666667,2.100000,10,2,10,0.0,0.0,0,1,49.0,112784
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,23,64,86,2.601932,0.084729,76,29.818182,3.739130,6,4,12,0.0,0.0,0,1,25.0,57210
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,7,14,18,0.704780,0.067780,7,121.000000,2.571429,12,2,2,0.0,0.0,0,1,24.0,138919
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,1,2,2,0.060983,0.030492,471,0.000000,2.000000,37,1,37,0.0,0.0,0,1,54.0,128309
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,6,12,13,0.469695,0.059305,41,134.000000,2.166667,12,2,3,1.0,1.0,0,2,52.0,52277


In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)
X_train_cid=X_train['customer_id']
X_test_cid=X_test['customer_id']
X_train=X_train.drop(['customer_id'],axis=1)
X_test=X_test.drop(['customer_id'],axis=1)
X_train.head()

,visits,no_unique_items_bought,total_items_bought,total_monetary_value,max_spent_on_item,recency,latency,avg_bought_per_visit,label,count,most_group,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
909308,4,22,25,0.819068,0.050831,618,25.333333,6.250000,12,3,49,0.0,0.0,0,1,27.0,132947
1029302,27,76,132,5.855441,0.167797,42,26.038462,4.888889,4,4,35,0.0,0.0,0,1,22.0,115024
1300670,11,21,22,1.062000,0.084729,57,58.200000,2.000000,10,2,10,1.0,1.0,0,2,31.0,245381
865176,5,9,10,0.343390,0.084729,275,93.750000,2.000000,22,2,40,1.0,1.0,0,2,56.0,326016
218963,3,8,8,0.193085,0.050831,197,150.500000,2.666667,22,1,23,1.0,1.0,0,2,44.0,65146


In [43]:
mlp = MLPClassifier(hidden_layer_sizes=(128,64,32), activation='relu', solver='adam', max_iter=500, lr=0.001)
mlp.fit(X_train,y_train)

MLPClassifier(hidden_layer_sizes=(128, 64, 32), max_iter=500)

In [44]:
pred_y = mlp.predict(X_test)
ac = accuracy_score(pred_y, y_test)
ac

0.0764377803523156